In [ ]:
from shutil import copyfile
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from PIL import Image
from tensorflow.keras import layers

In [ ]:
## train path => /kaggle/input/cassava-leaf-disease-classification/train_images
## train csv path => /kaggle/input/cassava-leaf-disease-classification/train.csv


## test images => /kaggle/input/cassava-leaf-disease-classification/test_images
trainFrame = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
trainFrame.label = trainFrame.label.astype(str)

mp1 = {'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

trainFrame.label = trainFrame.label.map(mp1)


data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.,
    validation_split=0.25,
)

train_data_loader = data_generator.flow_from_dataframe(
    trainFrame,
    directory= '/kaggle/input/cassava-leaf-disease-classification/train_images',
    x_col="image_id",
    y_col="label",
    target_size=(224, 224),
    subset='training'
)

val_data_loader = data_generator.flow_from_dataframe(
    trainFrame,
    directory= '/kaggle/input/cassava-leaf-disease-classification/train_images',
    x_col="image_id",
    y_col="label",
    target_size=(224, 224),
    subset='validation'
)




In [ ]:
## Defining Cnn
## lets define our CNN
### this model will not be used but other will
model =  tf.keras.models.Sequential([
    layers.BatchNormalization(),
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    layers.Conv2D(filters=512, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(5, activation='softmax')
])
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
## Transfer learning Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.models import Model

pretrained = InceptionResNetV2(include_top=True,weights='../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')

x=pretrained.layers[-2].output
fc1 = Dense(5,activation='softmax')(x)
model = Model(inputs=pretrained.input,outputs=fc1)

for l in model.layers[:-35]:
    l.trainable = False

    

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])





In [ ]:
def scheduler(epoch, lr):
    if epoch < 18:
        return lr
    else:
        return lr * 0.3

    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
retVal = model.fit(train_data_loader, validation_data=val_data_loader, callbacks=[callback], batch_size= 200, epochs= 20)

In [ ]:
plt.plot(retVal.history['loss'], label = 'training loss')
plt.plot(retVal.history['accuracy'], label = 'training accuracy')
plt.legend()

In [ ]:
plt.plot(retVal.history['val_loss'], label = 'validation loss')
plt.plot(retVal.history['val_accuracy'], label = 'validation accuracy')
plt.legend()

In [ ]:
frame = {'image_id' : [],
            'label' : []
           }

def predictor():
    path = '/kaggle/input/cassava-leaf-disease-classification/test_images'
    files = os.listdir(path)
    for mem in files:
        img = Image.open(path + '/' + mem)
        img = cv2.resize(np.asarray(img),(224, 224))
        img = np.array([img/255])
        snakeName = np.argmax (model.predict(img))
        frame['image_id'].append(mem)
        frame['label'].append (snakeName)
        #print (snakeName)

In [ ]:
predictor()

In [ ]:
frame = pd.DataFrame(frame)
frame.to_csv('submission.csv', index= False)

In [ ]:
frame